**Step_1: Get the data of 20 companies from yahoo web and store in google drive**

In [1]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd
#datetime is a Python module
import datetime
import os 

# Set the start and end date
start_date = '2017-01-01'

#start_date = '2017-01-01'
# Get data to today
today = datetime.datetime.today()
# Set the list including 20'str' acronym of 20 companies
list = ['INTC','AMD', 'CSCO', 'AAPL', 'MU', 'NVDA', 'QCOM', 'AMZN', 'NFLX',
        'FB', 'GOOG', 'BABA', 'EBAY', 'IBM', 'XLNX', 'TXN', 'NOK', 'TSLA',
        'MSFT', 'SNPS']

# Get the data
files=[]

# Create a data folder in my google drive.
def SaveData(df, filename):
    os.makedirs("./Dataset/", exist_ok = True)
    df.to_csv('./Dataset/'+filename+'.csv')

def GetData(acronym):
    print (acronym)
    data = pdr.get_data_yahoo(acronym, start = start_date, end = today )
    dataname= acronym
    files.append(dataname)
    SaveData(data, dataname)

print("Today is " + str(today))
#Get data, and save that data as "str".csv
for i in list:
    GetData(i)

Today is 2021-07-18 22:11:10.957716
INTC
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
CSCO
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
MU
[*********************100%***********************]  1 of 1 completed
NVDA
[*********************100%***********************]  1 of 1 completed
QCOM
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
NFLX
[*********************100%***********************]  1 of 1 completed
FB
[*********************100%***********************]  1 of 1 completed
GOOG
[*********************100%***********************]  1 of 1 completed
BABA
[*********************100%***********************]  1 of 1 completed
EBAY
[*********************100%***********************]  1 of 1 completed
IBM
[**

### Requirement: 
30 ngày liên tiếp là input đầu vào của cột Adj => Predict ra cho ngày 31 

**Step_2: Working with LSTM model**

In [4]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM
import numpy as np
import time
from sklearn.model_selection import train_test_split

AttributeError: type object 'TypeSpecProto' has no attribute 'NDARRAY_SPEC'

Read file csv

In [ ]:
data = pd.read_csv("/content/Dataset/GOOG.csv")
data

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
plt.figure(figsize=(16,6))
plt.title('Close Price History')
plt.plot(data['Adj Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Adj Close Price USD ($)', fontsize=18)
plt.show()

Split data train/test

In [ ]:
# Get norm
scaler = MinMaxScaler(feature_range=(0,1))

data_ = data.filter(['Close'])
scaled_data = scaler.fit_transform(data_)

scaled_data

In [ ]:
scaled_data.shape

In [ ]:
# Split the data into x_train and y_train data sets
X = []
y = []

for i in range(30, scaled_data.shape[0]):
    X.append(scaled_data[i-30:i, 0])
    y.append(scaled_data[i, 0])

Building model

In [ ]:
# Convert the x_train and y_train to numpy arrays 
X = np.array(X)
y = np.array(y)

# Reshape the data
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
X.shape

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
start = time.time()
model.compile(optimizer='adam', loss='mean_squared_error')
print("> Compilation Time : ", time.time() - start)
print(model.summary())

Training model

In [ ]:
history = model.fit(X_train, y_train, epochs=60, 
                    validation_data=(X_validate,y_validate), 
                    shuffle=True, batch_size=16, verbose=2)

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(loc='best')
plt.show()

**Step_3: Get results**

In [ ]:
Xt = model.predict(X)

In [ ]:
predict = []
for i in range(0,len(Xt)):
    predict.extend(scaler.inverse_transform(Xt[i].reshape(-1,1)))
predict

**Step_4: Check**

In [ ]:
plt.figure(figsize=(20, 12))

plt.subplot(211)
plt.title('Adj Close Price History', fontsize=18)
plt.ylabel('Adj Close Price USD ($)')
plt.plot(data['Adj Close'], color='y', label='True')
plt.legend(loc='best')

plt.subplot(212)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Adj Close Price USD ($)')
plt.plot(predict, color='r', label='Prediction')
plt.plot(scaler.inverse_transform(y.reshape(-1,1)), label='Target')
plt.legend(loc='best')

plt.show()